<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/SI_logistico_sazonal_cont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.integrate import solve_ivp

def si_logistic_edo(t, y, beta, epsilon, nu, mu, alpha, K):
    """
    Sistema de equações diferenciais para o modelo SI com crescimento logístico e sazonalidade.

    Parâmetros:
        t (float): Tempo (usado para sazonalidade).
        y (list): Lista com [S, I], onde:
            S - Indivíduos suscetíveis.
            I - Indivíduos infectados.
        beta, epsilon, nu, mu, alpha, K (float): Parâmetros do modelo.

    Retorna:
        [dS, dI]: Derivadas de S e I no tempo t.
    """
    S, I = y
    N = S + I  # População total

    # Evitar divisão por zero
    N = max(N, 1e-10)

    # Fator sazonal na taxa de transmissão
    sazonalidade = 1 + epsilon * np.cos(2 * np.pi * t / 365)

    # Equações diferenciais
    dS = N * (nu - N * (nu - mu) / K) - beta * sazonalidade * S * I / N - mu * S
    dI = beta * sazonalidade * S * I / N - (mu + alpha) * I

    return [dS, dI]

def simular_si_logistico(N0, I0, beta, epsilon, nu, mu, alpha, K, dt, T):
    """
    Resolve numericamente as equações diferenciais usando o método RKF45.

    Retorna:
        t_values: Vetor de tempo.
        N_values: Evolução da população total.
        I_values: Evolução do número de infectados.
    """
    S0 = N0 - I0  # População inicial de suscetíveis
    y0 = [S0, I0] # Estado inicial do sistema

    # Definição do tempo de simulação
    t_span = (0, T)
    t_eval = np.arange(0, T, dt)

    # Resolver EDOs com método Runge-Kutta-Fehlberg (RKF45)
    sol = solve_ivp(si_logistic_edo, t_span, y0, t_eval=t_eval, method='RK45',
                    args=(beta, epsilon, nu, mu, alpha, K))

    # Extraindo resultados
    t_values = sol.t
    S_values, I_values = sol.y
    N_values = S_values + I_values  # População total

    return t_values, N_values, I_values

def update_plot(beta, epsilon, nu, mu, alpha, N0, I0, dt, T, K):
    """
    Gera gráficos da simulação e os salva em alta resolução para artigos científicos.
    """
    t_values, N_values, I_values = simular_si_logistico(N0, I0, beta, epsilon, nu, mu, alpha, K, dt, T)

    plt.figure(figsize=(8, 6), dpi=100)

    # Gráfico de N(t)
    plt.subplot(2, 2, 1)
    plt.plot(t_values, N_values, linewidth=1.3)
    plt.xlabel('$t$ (dias)')
    plt.ylabel('$N(t)$')
    plt.grid(True)
    #plt.legend()
    plt.savefig("N_t.png", dpi=300)  # Salvar em alta resolução

    # Gráfico de I(t)
    plt.subplot(2, 2, 2)
    plt.plot(t_values, I_values, linewidth=1.3, c='red')
    plt.xlabel('$t$ (dias)')
    plt.ylabel('$I(t)$')
    plt.grid(True)
    #plt.legend()
    plt.savefig("I_t.png", dpi=300)  # Salvar em alta resolução

    # Diagrama de fase I x N
    plt.subplot(2, 2, 3)
    plt.plot(I_values, N_values, linewidth=1.3, c='black')
    plt.xlabel('$I(t)$')
    plt.ylabel('$N(t)$')
    plt.grid(True)
    plt.savefig("I_N_phase.png", dpi=300)  # Salvar em alta resolução

    plt.tight_layout()
    plt.show()

# Criando sliders interativos para ajustar os parâmetros
parametros = {
    'beta': widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.0001, description='beta:',readout_format='.4f'),
    'epsilon': widgets.FloatSlider(value=0.58, min=0.0, max=1.0, step=0.0001, description='epsilon:',readout_format='.4f'),
    'nu': widgets.FloatSlider(value=0.06, min=0.0, max=0.1, step=0.0001, description='nu:',readout_format='.4f'),
    'mu': widgets.FloatSlider(value=0.02, min=0.0, max=0.1, step=0.0001, description='mu:',readout_format='.4f'),
    'alpha': widgets.FloatSlider(value=0.04, min=0.0, max=0.1, step=0.0001, description='alpha:',readout_format='.4f'),
    'N0': widgets.IntSlider(value=10000, min=1000, max=100000, step=100, description='N0:',readout_format='.4f'),
    'I0': widgets.IntSlider(value=10, min=1, max=1000, step=1, description='I0:'),
    'dt': widgets.FloatSlider(value=0.1, min=0.001, max=1.0, step=0.001, description='dt:',readout_format='.4f'),
    'T': widgets.IntSlider(value=3*365, min=10, max=10*365, step=10, description='T:',readout_format='.4f'),
    'K': widgets.IntSlider(value=19500, min=1000, max=100000, step=100, description='K')
}

# Criando a interface interativa
iu = widgets.VBox(list(parametros.values()))
out = widgets.interactive_output(update_plot, parametros)

display(iu, out)


Output()